In [1]:
#  10/20 catboost

### Load lib

In [32]:
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, StratifiedKFold
from catboost import CatBoostClassifier

### Check data

In [33]:
data = pd.read_csv('../data/data_v2_19_oct.csv')

In [34]:
data.head()

,Unnamed: 0,pair_id,name_1,name_2,is_duplicate,name_1_upd,name_2_upd
0,0,1,Iko Industries Ltd.,"Enormous Industrial Trade Pvt., Ltd.",0,Iko,Enormous Trade
1,1,2,Apcotex Industries Ltd.,Technocraft Industries (India) Ltd.,0,Apcotex,Technocraft India
2,2,3,"Rishichem Distributors Pvt., Ltd.",Dsa,0,Rishichem Distributors,Dsa
3,3,4,Powermax Rubber Factory,Co. One,0,Powermax Factory,One
4,4,5,Tress A/S,Longyou Industries Park Zhejiang,0,Tress,Longyou Park Zhejiang


In [35]:
data.name_1_upd = data.name_1_upd.apply(lambda comp_name: ''.join([alpha if alpha.isalpha() or alpha==' ' else "" for alpha in comp_name]))
data.name_2_upd = data.name_2_upd.apply(lambda comp_name: ''.join([alpha if alpha.isalpha() or alpha==' ' else "" for alpha in comp_name]))

In [48]:
max_features = 550

In [49]:
skf = StratifiedKFold(n_splits=5)
f1_train_values = []
f1_test_values = []

for train_index, test_index in skf.split(data, data.is_duplicate):
    
    train, test = data.loc[train_index], data.loc[test_index]
    y_train, y_test = data.is_duplicate[train_index], data.is_duplicate[test_index]
    
    tfidf = TfidfVectorizer(max_df=0.85, min_df=10, max_features = max_features)
    data_to_tfidf = pd.concat([train.name_1_upd, train.name_2_upd])
    data_to_tfidf = data_to_tfidf.unique()
    
    
    tfidf.fit(data_to_tfidf)
    train_name_1 = pd.DataFrame(tfidf.transform(train.name_1_upd).toarray())
    train_name_2 = pd.DataFrame(tfidf.transform(train.name_2_upd).toarray())
    
    train_name_data = pd.concat([train_name_1, train_name_2], axis=1, ignore_index=False)
    train_name_data.columns = [i for i in range(max_features * 2)]
    
    model = CatBoostClassifier(task_type="GPU", random_state=1771)
    model.fit(train_name_data, train.is_duplicate)
    
    test_name_1 = pd.DataFrame(tfidf.transform(test.name_1_upd).toarray())
    test_name_2 = pd.DataFrame(tfidf.transform(test.name_2_upd).toarray())
    test_name_data = pd.concat([test_name_1, test_name_2], axis=1, ignore_index=False)
    test_name_data.columns = [i for i in range(max_features * 2)]
    
    f1_train = f1_score(train.is_duplicate, model.predict(train_name_data))
    f1_test = f1_score(test.is_duplicate, model.predict(test_name_data))
    
    
    
    f1_train_values.append(f1_train)
    f1_test_values.append(f1_test)
    
    


Learning rate set to 0.024253
0:	learn: 0.6334490	total: 14.7ms	remaining: 14.7s
1:	learn: 0.5787980	total: 28.9ms	remaining: 14.4s
2:	learn: 0.5278985	total: 43.5ms	remaining: 14.5s
3:	learn: 0.4819998	total: 59.7ms	remaining: 14.9s
4:	learn: 0.4405694	total: 75.5ms	remaining: 15s
5:	learn: 0.4021090	total: 92.9ms	remaining: 15.4s
6:	learn: 0.3669771	total: 109ms	remaining: 15.4s
7:	learn: 0.3357950	total: 125ms	remaining: 15.5s
8:	learn: 0.3075053	total: 141ms	remaining: 15.5s
9:	learn: 0.2819039	total: 157ms	remaining: 15.5s
10:	learn: 0.2586040	total: 172ms	remaining: 15.5s
11:	learn: 0.2377961	total: 188ms	remaining: 15.4s
12:	learn: 0.2188517	total: 204ms	remaining: 15.5s
13:	learn: 0.2019962	total: 222ms	remaining: 15.6s
14:	learn: 0.1866037	total: 238ms	remaining: 15.6s
15:	learn: 0.1726384	total: 254ms	remaining: 15.6s
16:	learn: 0.1605405	total: 269ms	remaining: 15.6s
17:	learn: 0.1492567	total: 286ms	remaining: 15.6s
18:	learn: 0.1387675	total: 301ms	remaining: 15.6s
19:	lea

165:	learn: 0.0214265	total: 2.65s	remaining: 13.3s
166:	learn: 0.0213964	total: 2.67s	remaining: 13.3s
167:	learn: 0.0213766	total: 2.69s	remaining: 13.3s
168:	learn: 0.0213549	total: 2.7s	remaining: 13.3s
169:	learn: 0.0213304	total: 2.72s	remaining: 13.3s
170:	learn: 0.0212960	total: 2.73s	remaining: 13.3s
171:	learn: 0.0212703	total: 2.75s	remaining: 13.2s
172:	learn: 0.0212447	total: 2.76s	remaining: 13.2s
173:	learn: 0.0212079	total: 2.78s	remaining: 13.2s
174:	learn: 0.0211764	total: 2.8s	remaining: 13.2s
175:	learn: 0.0211451	total: 2.81s	remaining: 13.2s
176:	learn: 0.0211176	total: 2.83s	remaining: 13.2s
177:	learn: 0.0210952	total: 2.85s	remaining: 13.1s
178:	learn: 0.0210646	total: 2.86s	remaining: 13.1s
179:	learn: 0.0210411	total: 2.88s	remaining: 13.1s
180:	learn: 0.0210119	total: 2.9s	remaining: 13.1s
181:	learn: 0.0209794	total: 2.91s	remaining: 13.1s
182:	learn: 0.0209550	total: 2.93s	remaining: 13.1s
183:	learn: 0.0209265	total: 2.95s	remaining: 13.1s
184:	learn: 0.0

324:	learn: 0.0187697	total: 5.25s	remaining: 10.9s
325:	learn: 0.0187557	total: 5.27s	remaining: 10.9s
326:	learn: 0.0187364	total: 5.28s	remaining: 10.9s
327:	learn: 0.0187250	total: 5.3s	remaining: 10.9s
328:	learn: 0.0187144	total: 5.31s	remaining: 10.8s
329:	learn: 0.0187052	total: 5.33s	remaining: 10.8s
330:	learn: 0.0186996	total: 5.34s	remaining: 10.8s
331:	learn: 0.0186828	total: 5.36s	remaining: 10.8s
332:	learn: 0.0186745	total: 5.37s	remaining: 10.8s
333:	learn: 0.0186659	total: 5.39s	remaining: 10.7s
334:	learn: 0.0186587	total: 5.4s	remaining: 10.7s
335:	learn: 0.0186480	total: 5.42s	remaining: 10.7s
336:	learn: 0.0186424	total: 5.44s	remaining: 10.7s
337:	learn: 0.0186337	total: 5.45s	remaining: 10.7s
338:	learn: 0.0186293	total: 5.47s	remaining: 10.7s
339:	learn: 0.0186253	total: 5.48s	remaining: 10.6s
340:	learn: 0.0186189	total: 5.5s	remaining: 10.6s
341:	learn: 0.0186106	total: 5.51s	remaining: 10.6s
342:	learn: 0.0185993	total: 5.53s	remaining: 10.6s
343:	learn: 0.0

489:	learn: 0.0175495	total: 7.87s	remaining: 8.19s
490:	learn: 0.0175450	total: 7.88s	remaining: 8.17s
491:	learn: 0.0175372	total: 7.9s	remaining: 8.15s
492:	learn: 0.0175314	total: 7.92s	remaining: 8.14s
493:	learn: 0.0175276	total: 7.93s	remaining: 8.12s
494:	learn: 0.0175242	total: 7.95s	remaining: 8.11s
495:	learn: 0.0175157	total: 7.96s	remaining: 8.09s
496:	learn: 0.0175046	total: 7.98s	remaining: 8.07s
497:	learn: 0.0174999	total: 7.99s	remaining: 8.06s
498:	learn: 0.0174976	total: 8.01s	remaining: 8.04s
499:	learn: 0.0174919	total: 8.03s	remaining: 8.03s
500:	learn: 0.0174887	total: 8.04s	remaining: 8.01s
501:	learn: 0.0174797	total: 8.06s	remaining: 8s
502:	learn: 0.0174772	total: 8.07s	remaining: 7.98s
503:	learn: 0.0174728	total: 8.09s	remaining: 7.96s
504:	learn: 0.0174663	total: 8.11s	remaining: 7.95s
505:	learn: 0.0174588	total: 8.12s	remaining: 7.93s
506:	learn: 0.0174505	total: 8.14s	remaining: 7.92s
507:	learn: 0.0174453	total: 8.16s	remaining: 7.9s
508:	learn: 0.017

655:	learn: 0.0167252	total: 10.5s	remaining: 5.49s
656:	learn: 0.0167232	total: 10.5s	remaining: 5.48s
657:	learn: 0.0167210	total: 10.5s	remaining: 5.46s
658:	learn: 0.0167181	total: 10.5s	remaining: 5.44s
659:	learn: 0.0167157	total: 10.5s	remaining: 5.43s
660:	learn: 0.0167129	total: 10.5s	remaining: 5.41s
661:	learn: 0.0167100	total: 10.6s	remaining: 5.39s
662:	learn: 0.0167081	total: 10.6s	remaining: 5.38s
663:	learn: 0.0167040	total: 10.6s	remaining: 5.36s
664:	learn: 0.0167012	total: 10.6s	remaining: 5.34s
665:	learn: 0.0166931	total: 10.6s	remaining: 5.33s
666:	learn: 0.0166894	total: 10.6s	remaining: 5.31s
667:	learn: 0.0166866	total: 10.7s	remaining: 5.29s
668:	learn: 0.0166835	total: 10.7s	remaining: 5.28s
669:	learn: 0.0166772	total: 10.7s	remaining: 5.26s
670:	learn: 0.0166747	total: 10.7s	remaining: 5.25s
671:	learn: 0.0166720	total: 10.7s	remaining: 5.23s
672:	learn: 0.0166691	total: 10.7s	remaining: 5.21s
673:	learn: 0.0166629	total: 10.7s	remaining: 5.2s
674:	learn: 0

...

270:	learn: 0.0187866	total: 4.72s	remaining: 12.7s
271:	learn: 0.0187764	total: 4.73s	remaining: 12.7s
272:	learn: 0.0187636	total: 4.75s	remaining: 12.7s
273:	learn: 0.0187445	total: 4.76s	remaining: 12.6s
274:	learn: 0.0187298	total: 4.78s	remaining: 12.6s
275:	learn: 0.0187207	total: 4.8s	remaining: 12.6s
276:	learn: 0.0187039	total: 4.82s	remaining: 12.6s
277:	learn: 0.0186932	total: 4.83s	remaining: 12.6s
278:	learn: 0.0186815	total: 4.85s	remaining: 12.5s
279:	learn: 0.0186679	total: 4.87s	remaining: 12.5s
280:	learn: 0.0186601	total: 4.88s	remaining: 12.5s
281:	learn: 0.0186399	total: 4.9s	remaining: 12.5s
282:	learn: 0.0186327	total: 4.92s	remaining: 12.5s
283:	learn: 0.0186217	total: 4.93s	remaining: 12.4s
284:	learn: 0.0186128	total: 4.95s	remaining: 12.4s
285:	learn: 0.0185980	total: 4.96s	remaining: 12.4s
286:	learn: 0.0185739	total: 4.98s	remaining: 12.4s
287:	learn: 0.0185629	total: 5s	remaining: 12.4s
288:	learn: 0.0185553	total: 5.01s	remaining: 12.3s
289:	learn: 0.018

436:	learn: 0.0170362	total: 7.31s	remaining: 9.42s
437:	learn: 0.0170296	total: 7.33s	remaining: 9.4s
438:	learn: 0.0170221	total: 7.34s	remaining: 9.38s
439:	learn: 0.0170161	total: 7.36s	remaining: 9.36s
440:	learn: 0.0170056	total: 7.37s	remaining: 9.34s
441:	learn: 0.0169998	total: 7.38s	remaining: 9.32s
442:	learn: 0.0169957	total: 7.4s	remaining: 9.3s
443:	learn: 0.0169923	total: 7.41s	remaining: 9.29s
444:	learn: 0.0169868	total: 7.43s	remaining: 9.27s
445:	learn: 0.0169763	total: 7.44s	remaining: 9.25s
446:	learn: 0.0169685	total: 7.46s	remaining: 9.23s
447:	learn: 0.0169635	total: 7.47s	remaining: 9.21s
448:	learn: 0.0169551	total: 7.49s	remaining: 9.19s
449:	learn: 0.0169434	total: 7.5s	remaining: 9.17s
450:	learn: 0.0169384	total: 7.52s	remaining: 9.15s
451:	learn: 0.0169322	total: 7.53s	remaining: 9.13s
452:	learn: 0.0169257	total: 7.55s	remaining: 9.11s
453:	learn: 0.0169151	total: 7.56s	remaining: 9.09s
454:	learn: 0.0169122	total: 7.58s	remaining: 9.08s
455:	learn: 0.01

601:	learn: 0.0160304	total: 9.91s	remaining: 6.55s
602:	learn: 0.0160279	total: 9.93s	remaining: 6.53s
603:	learn: 0.0160251	total: 9.94s	remaining: 6.52s
604:	learn: 0.0160229	total: 9.96s	remaining: 6.5s
605:	learn: 0.0160183	total: 9.97s	remaining: 6.48s
606:	learn: 0.0160155	total: 9.98s	remaining: 6.46s
607:	learn: 0.0160131	total: 10s	remaining: 6.45s
608:	learn: 0.0160066	total: 10s	remaining: 6.43s
609:	learn: 0.0160019	total: 10s	remaining: 6.42s
610:	learn: 0.0159991	total: 10.1s	remaining: 6.4s
611:	learn: 0.0159954	total: 10.1s	remaining: 6.38s
612:	learn: 0.0159936	total: 10.1s	remaining: 6.36s
613:	learn: 0.0159886	total: 10.1s	remaining: 6.35s
614:	learn: 0.0159826	total: 10.1s	remaining: 6.33s
615:	learn: 0.0159798	total: 10.1s	remaining: 6.31s
616:	learn: 0.0159773	total: 10.1s	remaining: 6.3s
617:	learn: 0.0159746	total: 10.2s	remaining: 6.28s
618:	learn: 0.0159722	total: 10.2s	remaining: 6.26s
619:	learn: 0.0159685	total: 10.2s	remaining: 6.25s
620:	learn: 0.0159642

761:	learn: 0.0153382	total: 12.6s	remaining: 3.92s
762:	learn: 0.0153363	total: 12.6s	remaining: 3.9s
763:	learn: 0.0153296	total: 12.6s	remaining: 3.89s
764:	learn: 0.0153278	total: 12.6s	remaining: 3.87s
765:	learn: 0.0153244	total: 12.6s	remaining: 3.85s
766:	learn: 0.0153205	total: 12.6s	remaining: 3.84s
767:	learn: 0.0153176	total: 12.7s	remaining: 3.82s
768:	learn: 0.0153160	total: 12.7s	remaining: 3.81s
769:	learn: 0.0153141	total: 12.7s	remaining: 3.79s
770:	learn: 0.0153104	total: 12.7s	remaining: 3.77s
771:	learn: 0.0153081	total: 12.7s	remaining: 3.76s
772:	learn: 0.0153039	total: 12.7s	remaining: 3.74s
773:	learn: 0.0152986	total: 12.8s	remaining: 3.72s
774:	learn: 0.0152955	total: 12.8s	remaining: 3.71s
775:	learn: 0.0152897	total: 12.8s	remaining: 3.69s
776:	learn: 0.0152881	total: 12.8s	remaining: 3.67s
777:	learn: 0.0152829	total: 12.8s	remaining: 3.66s
778:	learn: 0.0152760	total: 12.8s	remaining: 3.64s
779:	learn: 0.0152709	total: 12.9s	remaining: 3.63s
780:	learn: 0

924:	learn: 0.0148213	total: 15.2s	remaining: 1.23s
925:	learn: 0.0148169	total: 15.2s	remaining: 1.22s
926:	learn: 0.0148148	total: 15.2s	remaining: 1.2s
927:	learn: 0.0148117	total: 15.3s	remaining: 1.18s
928:	learn: 0.0148094	total: 15.3s	remaining: 1.17s
929:	learn: 0.0148050	total: 15.3s	remaining: 1.15s
930:	learn: 0.0148016	total: 15.3s	remaining: 1.13s
931:	learn: 0.0148002	total: 15.3s	remaining: 1.12s
932:	learn: 0.0147978	total: 15.3s	remaining: 1.1s
933:	learn: 0.0147950	total: 15.4s	remaining: 1.08s
934:	learn: 0.0147937	total: 15.4s	remaining: 1.07s
935:	learn: 0.0147918	total: 15.4s	remaining: 1.05s
936:	learn: 0.0147899	total: 15.4s	remaining: 1.04s
937:	learn: 0.0147881	total: 15.4s	remaining: 1.02s
938:	learn: 0.0147867	total: 15.4s	remaining: 1s
939:	learn: 0.0147830	total: 15.5s	remaining: 987ms
940:	learn: 0.0147781	total: 15.5s	remaining: 971ms
941:	learn: 0.0147772	total: 15.5s	remaining: 954ms
942:	learn: 0.0147741	total: 15.5s	remaining: 938ms
943:	learn: 0.014

In [50]:
f1_train_values = [round(i, 3) for i in f1_train_values]
f1_test_values = [round(i, 3) for i in f1_test_values]

In [51]:
print(f'Train: {f1_train_values}')
print(f'Test: {f1_test_values}') 
print(f'{np.mean(f1_train_values):.2f}')
print(f'{np.mean(f1_test_values):.2f}')

Train: [0.775, 0.795, 0.793, 0.798, 0.803]
Test: [0.756, 0.782, 0.788, 0.791, 0.763]
0.79
0.78
